In [1]:
!pip install -U albumentations ultralytics optuna huggingface_hub
!git clone https://github.com/sathishkumar67/ADIS.git
!mv /teamspace/studios/this_studio/ADIS/* /teamspace/studios/this_studio/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 197.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 125.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 222.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 206.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 93.7 MB/s eta 0:00:00
  Created wheel for sqlalchemy: filename=sqlalchemy-2.0.40-cp310-cp310-linux_x86_64.whl size=2158586 sha256=980c441a22acc9b5de0bcd7ddb02c566b3ad314bb6cf6a7feec09fdefdfa9f57
  Stored in directory: /home/zeus/.cache/pip/wheels/6c/ee/8d/34e2d0bc7efefbfb2c2a6752a3e5aa357330b04957c7aec35a
Successfully built sqlalchemy
Cloning into 'ADIS'...



In [2]:
# necessary imports
from __future__ import annotations
import os
import joblib
import optuna
from huggingface_hub import hf_hub_download
from utils import unzip_file
from model import YOLO11Model

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/teamspace/studios/this_studio/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
# Define the global variables
REPO_ID = "pt-sk/ADIS" 
FILENAME_IN_REPO = "dataset.zip"
LOCAL_DIR = os.getcwd()
TRAIN_PATH = f"{LOCAL_DIR}/dataset/train"
VAL_PATH = f"{LOCAL_DIR}/dataset/val"
TEST_PATH = f"{LOCAL_DIR}/dataset/test"
DATASET_PATH = f"{LOCAL_DIR}/{FILENAME_IN_REPO}"
REPO_TYPE = "dataset"
NUM_CLASSES = 10                                               
CLASSES = ['Cat', 'Cattle', 'Chicken', 'Deer', 'Dog', "Squirrel", 'Eagle', 'Goat', 'Rodents', 'Snake'] 
DATA_YAML_FILE = f"{LOCAL_DIR}/data.yaml"
MODEL_PATH = "yolo11n.pt"

In [4]:
# download the dataset and unzip it
hf_hub_download(repo_id=REPO_ID, filename=FILENAME_IN_REPO, repo_type=REPO_TYPE, local_dir=LOCAL_DIR)
unzip_file(DATASET_PATH, LOCAL_DIR)


# Get the number of CPU cores
num_cores = os.cpu_count()
print(f"Number of CPU cores: {num_cores}")

dataset.zip:   0%|          | 0.00/9.57G [00:00<?, ?B/s]

Could not set the permissions on the file '/teamspace/studios/this_studio/.cache/huggingface/download/v7Xt8H1GQ8W09J-6CSgzA0zuBH0=.d957c623096e6fa45d2927706b5d650c880bbad6352ec28015eb8af4f6b5777f.incomplete'. Error: [Errno 13] Permission denied: '/teamspace/studios/tmp_bd1a2057-6f88-46dc-b62f-c460aa2ac889'.
Continuing without setting permissions.
Unzipping: 100%|██████████| 9.60G/9.60G [00:56<00:00, 170MB/s]

Number of CPU cores: 16


In [5]:
# split paths for model
data_yaml = f"""
train: {TRAIN_PATH}
val: {VAL_PATH}
test: {TEST_PATH}

nc: {NUM_CLASSES}
names: {CLASSES}
"""

# write data yaml file
with open(DATA_YAML_FILE, "w") as file:
    file.write(data_yaml)
    print("data yaml file written!.............")

data yaml file written!.............


In [6]:
# Define the objective function
def objective(trial):
    
    # Define callback to report intermediate results
    def on_train_epoch_end(score, epoch):
        trial.report(score, step=epoch)  
        if trial.should_prune():
            raise optuna.TrialPruned()

    callbacks = {
        "on_train_epoch_end" : on_train_epoch_end
    }
    
    # Define hyperparameters using Optuna suggestions
    lr0 = trial.suggest_float("lr0", 1e-5, 1e-3, log=True)
    lrf = trial.suggest_float("lrf", 0.1, 1, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0.00001, 0.01, log=True)
    warmup_momentum = trial.suggest_float("warmup_momentum", 0.6, 0.9)
    momentum = trial.suggest_float("momentum", 0.8, 0.99)
    
    CONFIG_DICT = {
    "task": "detect",
    "mode": "train",
    "bohb": True,
    "custom_callbacks": callbacks,
    "data": DATA_YAML_FILE,
    "batch": 768,
    "imgsz": 320,
    "save": True,
    "device": 0,
    "workers": num_cores,
    "pretrained": True,
    "optimizer": "AdamW",
    "seed": 42,
    "epochs": 10,
    "warmup_epochs": 2,
    "patience": 2}

    # Train YOLO model
    model = YOLO11Model(MODEL_PATH)
    model.train(**CONFIG_DICT, lr0=lr0, lrf=lrf, momentum=momentum, weight_decay=weight_decay, warmup_momentum=warmup_momentum)
    
    # Return validation mAP as the objective value
    return model.score 

In [ ]:
# Define the study
NUM_TRIALS = 55
study = optuna.create_study(direction='maximize', 
                            sampler=optuna.samplers.TPESampler(), 
                            pruner=optuna.pruners.HyperbandPruner(),
                            study_name="yolo11_tuning",
                            load_if_exists=True)

# Optimize with a callback to stop after NUM_TRIALS complete trials
study.optimize(
    objective,
    n_trials=NUM_TRIALS)

[I 2025-03-27 18:06:56,460] A new study created in memory with name: yolo11_tuning


Ultralytics 8.3.97 🚀 Python-3.10.10 torch-2.2.1+cu121 CUDA:0 (NVIDIA L40S, 45596MiB)
trainer: task=detect, mode=train, model=yolo11n.pt, data=/teamspace/studios/this_studio/data.yaml, epochs=10, time=None, patience=2, batch=704, imgsz=320, save=True, save_period=-1, cache=False, device=0, workers=16, project=None, name=train, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, s

100%|██████████| 755k/755k [00:00<00:00, 108MB/s]

Overriding model.yaml nc=80 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  blocks.Conv                                  [3, 16, 3, 2]                 
  1                  -1  1      4672  blocks.Conv                                  [16, 32, 3, 2]                
  2                  -1  1      6640  blocks.C3k2                                  [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  blocks.Conv                                  [64, 64, 3, 2]                
  4                  -1  1     26080  blocks.C3k2                                  [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  blocks.Conv                                  [128, 128, 3, 2]              
  6                  -1  1     87040  blocks.C3k2                                  [128, 128, 1, True]           
  7                  -1  1    295424  blocks.Con

 23        [16, 19, 22]  1    753262  blocks.Detect                                [10, [64, 128, 256]]          
YOLO11n summary: 239 layers, 2,912,430 parameters, 2,912,414 gradients, 7.7 GFLOPs

Transferred 421/463 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train', view at http://localhost:6006/
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅


...

                   all       1400       1781      0.748      0.689      0.757      0.522
Epoch 5: AVG Val Box Loss: 1.0909 | AVG Val Cls Loss: 1.2729 | AVG Val DFL Loss: 1.1412 | Total Val Loss: 3.5050







      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       6/10      23.1G      1.016       1.11      1.163        970        320:  48%|████▊     | 14/29 [00:07<00:08,  1.70it/s]libpng warning: iCCP: known incorrect sRGB profile
       6/10      23.1G      1.008      1.087      1.164        357        320: 100%|██████████| 29/29 [00:16<00:00,  1.75it/s]

Epoch 6: AVG Box Loss: 1.0078 | AVG Cls Loss: 1.0873 | AVG DFL Loss: 1.1637



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [ ]:
joblib.dump(study, "optuna_study.pkl")